In [ ]:
!pip install pulp
!pip install skggm
from sklearn.covariance import GraphicalLasso as glasso
import numpy as np
from scipy.io import loadmat
import pulp
from numba import jit, cuda
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.spatial.distance import cosine
from sklearn.metrics import f1_score
from scipy.sparse import csr_matrix
from time import time 
from six.moves import xrange
from inverse_covariance import QuicGraphicalLasso
import networkx as nx
from sklearn import preprocessing
from scipy.optimize import linprog
from cvxopt import matrix
from cvxopt import glpk
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigs
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.linear_model import LogisticRegression
#change 'sklearn.utils.testing' to 'sklearn.utils._testing
#Add:
#import joblib
#sys.modules['sklearn.externals.joblib'] = joblib

import pickle
def save_variable(variable,filename):
  pickle.dump(variable,open(filename, "wb"))
def load_variable(filename):
  return pickle.load(open(filename,'rb')) 

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))  

In [ ]:
def UpdateTheta(X,alpha):
  glasso_out = QuicGraphicalLasso(lam=alpha, tol=1e-0).fit(X)
  theta = glasso_out.precision_
  return theta

def UpdateC(A,D):
  Lp_prob = pulp.LpProblem('UpdateCe', pulp.LpMaximize)  
  var_names = ['var_' + str(i) for i in range(np.size(A,0))]
  C = [pulp.LpVariable(i, lowBound=0, upBound=1) for i in var_names]
  Lp_prob += 2*np.sum(abs(A)@C)
  ones_mat = np.ones((N,N))
  for i in range(N):
    for j in range(N):
      Lp_prob += -(C[i]+C[j]) >= -1-e*logD[i,j] # Constraints
  Lp_prob += (np.sum(C) == N/8) # Constraints

  
  status = Lp_prob.solve()   # Solver
  C_out = np.zeros((N,))
  for i in range(N):
   C_out[i] = C[i].varValue

  return C_out  

In [ ]:
#Loading data:

#C.elegans
data = loadmat('/Datasets/Celegans.mat')
A0=np.array(data['A0'].todense())
N=len(A0[:,0])
coords = data['coords']
X = data['coords'].T
logD = np.zeros((N,N))
for i in range(N):
  for j in range(N):
    logD[i,j] = 0.09*np.log(1000*(np.linalg.norm(coords[i,0:2]-coords[j,0:2])**2+0.001))

'''
#London underground
data = loadmat('/Datasets/London_underground.mat')
A0=np.array(data['A0'])
N=len(A0[:,0])
coords = data['Coords']
X = data['Coords'].T
logD = np.zeros((N,N))
for i in range(N):
  for j in range(N):
    logD[i,j] = 0.09*np.log(1000*(np.linalg.norm(coords[i,0:2]-coords[j,0:2])**2+0.001))

# Synthetic data
A0 = load_variable('/Datasets/syn/theta_syn')
N = np.size(A0,0)
X = load_variable('/Datasets/syn/X_ao')
D = np.size(X,1)
elogD = load_variable('/Datasets/syn/elogD_syn')    
c_true = load_variable('/Datasets/syn/c_syn')   
'''

In [ ]:
C = 0.1*np.ones((N,)) # initialize core scores
e = 0.1
alpha = 0.1
max_iter = 30
alpha_ = np.ones((N,N))

for m in range (0,max_iter):   
  #Update Theta-----------------------------------------------------------------
    for i in range (0,N):
      for j in range (i,N):
        alpha_aux = alpha*(1-C[i]-C[j]+e*logD[i,j]) 
        alpha_[i,j] = alpha_aux
        alpha_[j,i] = alpha_aux  
    theta = UpdateTheta(X, alpha_)
    A = np.diag(np.diag(theta))-theta
    print("A=",A)

  #Update C---------------------------------------------------------------------
    C = UpdateC(theta,logD)
    print("C=",C)